In [1]:
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split

caltech_dir = "./img"
categories = ['a', 'b', 'c']
nb_classes = len(categories)

image_w = 64
image_h = 64

pixels = image_h * image_w * 3

X = []
y = []

for idx, cat in enumerate(categories):
    
    #one-hot 돌리기.
    label = [0 for i in range(nb_classes)]
    label[idx] = 1

    image_dir = caltech_dir + "/" + cat
    files = glob.glob(image_dir+"/*.jpg")
    print(cat, " 파일 길이 : ", len(files))
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)

        X.append(data)
        y.append(label)

        if i % 700 == 0:
            print(cat, " : ", f)

X = np.array(X)
y = np.array(y)


X_train, X_test, y_train, y_test = train_test_split(X, y)
xy = (X_train, X_test, y_train, y_test)
np.save("./img/image_data.npy", xy)

print("ok", len(y))

a  파일 길이 :  218
a  :  ./img/a\HF010011_0000_0011.jpg
b  파일 길이 :  280
b  :  ./img/b\HF010012_0000_0001.jpg
c  파일 길이 :  390
c  :  ./img/c\HF010014_0000_0002.jpg
ok 888


C:\Users\BIT-R45\anaconda3\envs\project_env\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


In [2]:
import os, glob, numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

import tensorflow as tf

np_load_old = np.load
np.load = lambda *a, **k: np_load_old(*a, allow_pickle=True, **k)
X_train, X_test, y_train, y_test = np.load('./img/image_data.npy')
np.load = np_load_old

print(X_train.shape)
print(X_train.shape[0])

(666, 64, 64, 3)
666


In [3]:
categories = ['a', 'b', 'c']
nb_classes = len(categories)

X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255

In [4]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding="same", input_shape=X_train.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
    
model.add(Conv2D(64, (3,3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
    
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(nb_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0

In [6]:
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
21/21 [==============================] - 2s 107ms/step - loss: 1.6045 - accuracy: 0.5631 - val_loss: 0.8922 - val_accuracy: 0.7117
Epoch 2/50
21/21 [==============================] - 2s 98ms/step - loss: 0.4621 - accuracy: 0.8258 - val_loss: 0.3100 - val_accuracy: 0.8649
Epoch 3/50
21/21 [==============================] - 2s 97ms/step - loss: 0.2384 - accuracy: 0.9249 - val_loss: 0.2026 - val_accuracy: 0.9279
Epoch 4/50
21/21 [==============================] - 2s 99ms/step - loss: 0.1422 - accuracy: 0.9565 - val_loss: 0.1665 - val_accuracy: 0.9459
Epoch 5/50
21/21 [==============================] - 2s 98ms/step - loss: 0.0899 - accuracy: 0.9670 - val_loss: 0.1165 - val_accuracy: 0.9730
Epoch 6/50
21/21 [==============================] - 2s 106ms/step - loss: 0.0748 - accuracy: 0.9685 - val_loss: 0.0784 - val_accuracy: 0.9775
Epoch 7/50
21/21 [==============================] - 2s 99ms/step - loss: 0.0606 - accuracy: 0.9745 - val_loss: 0.0571 - val_accuracy: 0.9865
Epoch 8/50


In [7]:
print("정확도 : %.4f" % (model.evaluate(X_test, y_test)[1]))

7/7 [==============================] - 0s 18ms/step - loss: 0.0371 - accuracy: 0.9910
정확도 : 0.9910


In [8]:
from tensorflow.keras.models import load_model

model.save('model_sample.h5')

In [10]:
from tensorflow.keras.models import load_model

caltech_dir = "./test"
image_w = 64
image_h = 64

pixels = image_h * image_w * 3


X = []
filenames = []
files = glob.glob(caltech_dir+"/*.*")
for i, f in enumerate(files):
    img = Image.open(f)
    img = img.convert("RGB")
    img = img.resize((image_w, image_h))
    data = np.asarray(img)
    filenames.append(f)
    X.append(data)

X = np.array(X)
model = load_model('./model_sample.h5')

prediction = model.predict(X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
cnt = 0

#이 비교는 그냥 파일들이 있으면 해당 파일과 비교. 카테고리와 함께 비교해서 진행하는 것은 _4 파일.
for i in prediction:
    pre_ans = i.argmax()  # 예측 레이블
    print(i)
    print(pre_ans)
    pre_ans_str = ''
    if pre_ans == 0: pre_ans_str = "a"
    elif pre_ans == 1: pre_ans_str = "b"
    elif pre_ans == 2: pre_ans_str = "c"
    else: pre_ans_str = "게"
    if i[0] >= 0.8 : print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    if i[1] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"으로 추정됩니다.")
    if i[2] >= 0.8: print("해당 "+filenames[cnt].split("\\")[1]+"이미지는 "+pre_ans_str+"로 추정됩니다.")
    cnt += 1
    # print(i.argmax()) #얘가 레이블 [1. 0. 0.] 이런식으로 되어 있는 것을 숫자로 바꿔주는 것.
    # 즉 얘랑, 나중에 카테고리 데이터 불러와서 카테고리랑 비교를 해서 같으면 맞는거고, 아니면 틀린거로 취급하면 된다.
    # 이걸 한 것은 _4.py에.

[1.000 0.000 0.000]
0
해당 경릉.jpg이미지는 a로 추정됩니다.
[0.000 0.000 1.000]
2
해당 다운로드 (1).jpg이미지는 c로 추정됩니다.
[0.000 0.000 1.000]
2
해당 다운로드.jpg이미지는 c로 추정됩니다.
[1.000 0.000 0.000]
0
해당 얘릉.jpg이미지는 a로 추정됩니다.
[0.000 1.000 0.000]
1
해당 예릉1.jpg이미지는 b으로 추정됩니다.
